In [1]:
from RegistrationObj import RegistrationObj, IterativePairwise
import scipy.stats as st
from utils import colour_map, min_max_normalisation, mae, mae_permutation, pcd_to_tif, visualise_pcd
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from open3d import JVisualizer
import tifffile.tifffile as tiff
import copy 
%matplotlib notebook
import glob
import pandas as pd

* opened points, coloured total point cloud using ablation points as marker, saved as intensity file

In [2]:
def colour_region(total_points, region_points):
    
    total_points=total_points.to_numpy(dtype="float64")
    region_points=region_points.to_numpy(dtype="float64")

    region_intensity= np.zeros([len(total_points),1])

    for index in range(len(region_points)):
        corresponding_x=np.where(total_points[:,0]==region_points[index,0])
        corresponding_y=np.where(total_points[:,1]==region_points[index,1])
        corresponding_z=np.where(total_points[:,2]==region_points[index,2])
        for num_x in range(len(corresponding_x)):
            for num_y in range(len(corresponding_y)):
                for num_z in range(len(corresponding_z)):
                    if num_x==num_y==num_z:
                        total_pts_index=list(corresponding_x)[0][0]
                        region_intensity[total_pts_index]=1.0
            
    return region_intensity

In [3]:
def open_points_xls(files_path, cols=[0,1,2],row_skip=0, header=0):
   
    
    files_list = list(np.sort(glob.glob(files_path)))
    print(files_list)
    files_dataframe = [pd.read_csv(file, skiprows = [row_skip], header = header, usecols = cols) for file in files_list]
    
    return files_dataframe, files_list

In [4]:
tailbud_pts, tailbud_pts_path=open_points_xls("./datasets/7-ablation_alignment/xyz*.csv")
ablation_pts,ablation_pts_path=open_points_xls("./datasets/7-ablation_alignment/abl*.csv")

['./datasets/7-ablation_alignment/xyz_exp12_em07.csv', './datasets/7-ablation_alignment/xyz_exp12_em09.csv', './datasets/7-ablation_alignment/xyz_exp12_em10.csv']
['./datasets/7-ablation_alignment/abl_exp12_em07.csv', './datasets/7-ablation_alignment/abl_exp12_em09.csv', './datasets/7-ablation_alignment/abl_exp12_em10.csv']


In [5]:
tailbud_pts[0].shape

(1136, 3)

In [6]:
ablation_pts[0].shape

(89, 3)

In [7]:
em07_int=colour_region(tailbud_pts[0],ablation_pts[0])
em09_int=colour_region(tailbud_pts[1],ablation_pts[1])
em10_int=colour_region(tailbud_pts[2],ablation_pts[2])

print(sum(em07_int)==ablation_pts[0].shape[0],sum(em09_int)==ablation_pts[1].shape[0],sum(em10_int)==ablation_pts[2].shape[0])


[False] [ True] [ True]


In [8]:
em09_int.shape

(1115, 1)

In [9]:
pd.DataFrame(em07_int).to_csv('./datasets/7-ablation_alignment/col_abl_exp12_em07.csv', index = False)
pd.DataFrame(em09_int).to_csv('./datasets/7-ablation_alignment/col_abl_exp12_em09.csv', index = False)
pd.DataFrame(em10_int).to_csv('./datasets/7-ablation_alignment/col_abl_exp12_em10.csv', index = False)


In [10]:
pos_path_source_list = ["./datasets/7-ablation_alignment/xyz_exp12_em09.csv"]

abl_color_path_source_list = ["./datasets/7-ablation_alignment/col_abl_exp12_em09.csv"]

pos_path_target = "./datasets/7-ablation_alignment/xyz_exp12_em10.csv"

abl_color_path_target = "./datasets/7-ablation_alignment/col_abl_exp12_em10.csv"

"""
other_source_channels = [["./datasets/7-ablation_alignment/abl_col_exp12_em09.xls"],
                         ["./datasets/7-ablation_alignment/abl_col_exp12_em10.xls"]]

other_target_channels = [["./datasets/7-ablation_alignment/abl_col_exp11_em07.xls"]]
"""

'\nother_source_channels = [["./datasets/7-ablation_alignment/abl_col_exp12_em09.xls"],\n                         ["./datasets/7-ablation_alignment/abl_col_exp12_em10.xls"]]\n\nother_target_channels = [["./datasets/7-ablation_alignment/abl_col_exp11_em07.xls"]]\n'

In [11]:
abl_pts = IterativePairwise(pos_path_source_list, pos_path_target, abl_color_path_source_list, abl_color_path_target,
                             algorithm = "icp",pos_skiprows = [0], pos_usecols = [0,1,2], color_skiprows = [0], color_usecols = [0])


In [12]:
abl_pcd = abl_pts.iterative_registration(jupyter_visualise = False, verbose = False)

--- Registering Source dataset 0
--No other source channels detected
.csv - Successfully read files
[[122.71  20.54   1.  ]
 [ 96.13  21.97   1.  ]
 [116.95  21.8    1.  ]
 ...
 [ 51.69 105.69  41.  ]
 [ 51.31 105.54  41.  ]
 [ 51.81 112.98  41.  ]]
RegistrationResult with fitness=5.063291e-02, inlier_rmse=9.873058e+00, and correspondence_set size of 12
Access transformation to get result.
.csv - Successfully read files
[[122.71  20.54   1.  ]
 [ 96.13  21.97   1.  ]
 [116.95  21.8    1.  ]
 ...
 [ 51.69 105.69  41.  ]
 [ 51.31 105.54  41.  ]
 [ 51.81 112.98  41.  ]]
RegistrationResult with fitness=4.762332e-01, inlier_rmse=5.575640e+00, and correspondence_set size of 531
Access transformation to get result.


IndexError: index 1114 is out of bounds for axis 0 with size 1114

In [109]:
pos_path_source="./datasets/7-ablation_alignment/xyz_exp12_em09.csv"
pos_path_target="./datasets/7-ablation_alignment/xyz_exp12_em07.csv"
colour_path_source="./datasets/7-ablation_alignment/col_abl_exp12_em09.csv"
colour_path_target="./datasets/7-ablation_alignment/col_abl_exp12_em07.csv"

In [140]:
test=RegistrationObj(pos_path_source,pos_path_target,colour_path_source,colour_path_target,'ransac')

In [159]:
test.pos_skiprows=0
test.pos_usecols=[0,1,2]


In [181]:
rgb_array,lut=colour_map(normed, 'viridis')

In [184]:
rgb_array,lut

(array([[0.267004, 0.004874, 0.329415],
        [0.267004, 0.004874, 0.329415],
        [0.267004, 0.004874, 0.329415],
        ...,
        [0.267004, 0.004874, 0.329415],
        [0.267004, 0.004874, 0.329415],
        [0.267004, 0.004874, 0.329415]]),
 <matplotlib.cm.ScalarMappable at 0x7fed0e27a370>)

In [190]:
test.source_color[434:448]
normed=min_max_normalisation(test.source_color)
normed

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [162]:
source,target=test.create_pcd()

RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

In [146]:
source_processed, target_processed, source_fpfh, target_fpfh=test.preprocessing_func()

RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

In [117]:
RegistrationObj.__init__(pos_path_source,pos_path_target,colour_path_source,colour_path_target,'coloredICP')

AttributeError: 'str' object has no attribute 'pos_path_source'